### Check original format

In [1]:
import pickle
import gzip


def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

# original_dataset = load_dataset_file("/home/pdalbianco/Github/Signformer/PHOENIX2014T/phoenix14t.pami0.dev")
original_dataset = load_dataset_file("/mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.test")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.test'

In [ ]:
len(original_dataset)

5000

In [2]:
from slt_datasets.SLTDataset import SLTDataset


DATASET = "RWTH_PHOENIX_2014T"
# DATASET = "ISL"
DATA_DIR = "/mnt/disk3Tb/slt-datasets/"
INPUT_MODE = "pose"
OUTPUT_MODE = "text"
OUTPUT_DIR = "/mnt/disk3Tb/exported-slt-datasets"

### Export

In [3]:
import pickle
import torch
import gzip
from tqdm import tqdm
from slt_datasets.SLTDataset import SLTDataset


def format_pose(pose):
	pose = pose[:, 0, :, :2]
	return pose.reshape(pose.shape[0], -1)

def export_dataset(dataset: SLTDataset, output_path: str):
	samples = []
	for i in tqdm(range(len(dataset))):
		pose = format_pose(dataset[i][0])
		# pose is pytorch tensor shape (N, 1086), remove items from position 34 to 95 (face keypoints) to make it (N, 1024)
		pose = torch.cat((pose[:, :34], pose[:, 96:]), dim=1)
		pose = torch.nan_to_num(pose, nan=0.0)
		text = dataset.get_item_raw(i)[1]
		name = dataset.annotations.iloc[i]['id']
		samples.append({
			"sign": pose,
			"text": text,
			"gloss": "",
			"signer": "",
			"name": name
		})
	print(f"Saving dataset to {output_path}")
	with gzip.open(output_path, "wb") as f:
		pickle.dump(samples, f)		
	print(f"Dataset saved to {output_path}")

In [4]:
for SPLIT in ["train", "val", "test"]:
	dataset = SLTDataset(
		data_dir=DATA_DIR + DATASET,
		input_mode=INPUT_MODE,
		output_mode=OUTPUT_MODE,
		split=SPLIT
	)
	export_dataset(dataset, f"{OUTPUT_DIR}/{DATASET}.pami0.{SPLIT}")	

Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded train annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


Validating files: 100%|██████████| 7096/7096 [00:00<00:00, 23226.86it/s]


Dataset loaded correctly



100%|██████████| 7096/7096 [02:06<00:00, 56.05it/s]


Saving dataset to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.train
Dataset saved to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.train
Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded val annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


...

Saving dataset to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.val
Dataset saved to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.val
Loaded metadata for dataset: RWTH-PHOENIX-Weather 2014 T: Parallel Corpus of Sign Language Video, Gloss and Translation
Loaded test annotations at /mnt/disk3Tb/slt-datasets/RWTH_PHOENIX_2014T/annotations.csv


Validating files: 100%|██████████| 642/642 [00:00<00:00, 34597.75it/s]


Dataset loaded correctly



100%|██████████| 642/642 [00:11<00:00, 57.17it/s]


Saving dataset to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.test
Dataset saved to /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.test


### ISL has no anottations

In [2]:
# ------------------------------------------------------
# utils/export.py  (put it wherever you prefer)
# ------------------------------------------------------
import os
import gzip
import pickle
from pathlib import Path
from tqdm import tqdm
import torch


def format_pose(pose):
    pose = pose[:, 0, :, :2]
    return pose.reshape(pose.shape[0], -1)


def _write_chunk(chunk, out_path, part_idx):
    """Pickle-gzip one chunk to …/file.part-<idx>"""
    part_path = f"{out_path}.part-{part_idx}"
    Path(part_path).parent.mkdir(parents=True, exist_ok=True)
    with gzip.open(part_path, "wb") as f:
        pickle.dump(chunk, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"  ✔ saved {len(chunk):5d} samples → {part_path}")


def export_subset(dataset, indices, output_path, *, chunk_size=5_000):
    """
    Write <indices> to <output_path>.part-* in chunks of <chunk_size>.
    The file format of every part is exactly the same you used before
    (a pickled Python list of dicts).
    """
    buffer, part = [], 0
    print(f"Exporting {len(indices)} samples → {output_path}.part-* "
          f"(chunk_size={chunk_size})")

    for i in tqdm(indices):
        pose = format_pose(dataset[i][0])
        pose = torch.cat((pose[:, :34], pose[:, 96:]), dim=1)
        pose = torch.nan_to_num(pose, nan=0.0)

        text = dataset.get_item_raw(i)[1]
        name = dataset.annotations.iloc[i]["id"]

        buffer.append(
            {"sign": pose, "text": text, "gloss": "", "signer": "", "name": name}
        )

        if len(buffer) == chunk_size:
            _write_chunk(buffer, output_path, part)
            part += 1
            buffer = []

    if buffer:  # last, possibly smaller chunk
        _write_chunk(buffer, output_path, part)

In [3]:
import numpy as np


# ------------------------------------------------------------------
# 1.  Load the full ISL dataset (no split argument)
# ------------------------------------------------------------------
DATASET = "ISL"

isl_full = SLTDataset(
    data_dir      = DATA_DIR + DATASET,
    input_mode    = INPUT_MODE,
    output_mode   = OUTPUT_MODE,   # <- no split keyword passed
)

# ------------------------------------------------------------------
# 2.  Create a reproducible shuffled index list
# ------------------------------------------------------------------
rng = np.random.default_rng(seed=42)          # fixed seed = reproducible
shuffled_idx = rng.permutation(len(isl_full))

# 70 / 20 / 10  split points
n_total     = len(shuffled_idx)
cut1        = int(0.70 * n_total)
cut2        = int(0.90 * n_total)             # 70 % + 20 % = 90 %

idx_train   = shuffled_idx[:cut1]
idx_test    = shuffled_idx[cut1:cut2]
idx_val     = shuffled_idx[cut2:]

print(f"Total: {n_total}  ➜  train {len(idx_train)}  "
      f"test {len(idx_test)}  val {len(idx_val)}")

# ------------------------------------------------------------------
# 4.  Export each split
# ------------------------------------------------------------------

export_subset(isl_full, idx_val,
              f"{OUTPUT_DIR}/{DATASET}.pami0.val")
export_subset(isl_full, idx_test,
              f"{OUTPUT_DIR}/{DATASET}.pami0.test")
export_subset(isl_full, idx_train,
              f"{OUTPUT_DIR}/{DATASET}.pami0.train")

Loaded metadata for dataset: ISLTranslate
Loaded  annotations at /mnt/disk3Tb/slt-datasets/ISL/annotations.csv


Validating files: 100%|██████████| 125856/125856 [00:06<00:00, 18408.08it/s] 


Missing 17534 files out of 125856 (13.93%)

Total: 108322  ➜  train 75825  test 21664  val 10833
Exporting 10833 samples → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val.part-* (chunk_size=5000)


 46%|████▌     | 5001/10833 [06:36<37:27:33, 23.12s/it]

  ✔ saved  5000 samples → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val.part-0


...

  ✔ saved  5000 samples → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.train.part-13


 99%|█████████▉| 75004/75825 [1:40:22<3:24:41, 14.96s/it] 

  ✔ saved  5000 samples → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.train.part-14


100%|██████████| 75825/75825 [1:40:58<00:00, 12.52it/s]  


  ✔ saved   825 samples → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.train.part-15


In [4]:
# utils/concat.py
import glob
import gzip
import pickle
from pathlib import Path
from tqdm import tqdm

def merge_dataset_parts(prefix: str,
                        output_path: str,
                        *,
                        buffer_limit: int = 5_000) -> None:
    """
    Concatenate every  <prefix>.part-*  into one gzip file at <output_path>
    WITHOUT loading the whole dataset into memory.

    • buffer_limit : number of samples kept in RAM before they get written out.
    """
    part_paths = sorted(glob.glob(f"{prefix}.part-*"))
    if not part_paths:
        raise FileNotFoundError(f"No parts found for prefix '{prefix}'")

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    buffer = []

    print(f"Writing {len(part_paths)} parts → {output_path}")
    with gzip.open(output_path, "wb") as fout:
        for p in tqdm(part_paths, desc="Merging"):
            with gzip.open(p, "rb") as fpart:
                part_samples = pickle.load(fpart)        # one chunk
            buffer.extend(part_samples)

            if len(buffer) >= buffer_limit:
                pickle.dump(buffer, fout, protocol=pickle.HIGHEST_PROTOCOL)
                buffer.clear()                          # free RAM

        # last, possibly smaller remainder
        if buffer:
            pickle.dump(buffer, fout, protocol=pickle.HIGHEST_PROTOCOL)

    print("Done ✔")


def load_dataset_file(path: str):
    """
    Load a (possibly multi-pickle) gzip file produced by merge_dataset_parts.
    Works for the original one-pickle files as well.
    """
    samples = []
    with gzip.open(path, "rb") as f:
        while True:
            try:
                chunk = pickle.load(f)
            except EOFError:
                break
            samples.extend(chunk)
    return samples

In [ ]:
# ------------------------------------------------------------------
# Step 1:  Merge all parts of one split
# ------------------------------------------------------------------
merge_dataset_parts("/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val",
                    "/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val",
                    buffer_limit=5_000)      # keep ≤ 5 k samples in RAM

# ------------------------------------------------------------------
# Step 2:  Load the merged file later
# ------------------------------------------------------------------
train_samples = load_dataset_file(
        "/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val")

print(len(train_samples))         # 75 825
print(train_samples[0].keys())    # dict keys unchanged

Writing 5 parts → /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.test


Merging:   0%|          | 0/5 [00:00<?, ?it/s]

Merging: 100%|██████████| 5/5 [16:44<00:00, 200.89s/it]


Done ✔
21664
dict_keys(['sign', 'text', 'gloss', 'signer', 'name'])


In [8]:
# utils/concat_raw.py
import glob
import shutil
from pathlib import Path
from typing import Sequence

def concat_gzip_parts(prefix: str,
                      output_path: str,
                      *,
                      chunk_size: int = 512 * 1024) -> None:
    """
    Concatenate every <prefix>.part-* into a single gzip file
    without decompressing anything.

    Each .part-* must be a complete gzip member (which is true for your exporter).
    """
    part_paths: Sequence[str] = sorted(glob.glob(f"{prefix}.part-*"))
    if not part_paths:
        raise FileNotFoundError(f"No parts found for prefix '{prefix}'")

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)

    print(f"Concatenating {len(part_paths)} parts → {output_path}")
    with open(output_path, "wb") as fout:
        for path in part_paths:
            with open(path, "rb") as fin:
                shutil.copyfileobj(fin, fout, length=chunk_size)
    print("Done ✔")

concat_gzip_parts("/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.test", "/mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.test.gz")
concat_gzip_parts("/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val", "/mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.val.gz")
concat_gzip_parts("/mnt/disk3Tb/exported-slt-datasets/ISL.pami0.train", "/mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.train.gz")

Concatenating 5 parts → /mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.test.gz
Done ✔
Concatenating 3 parts → /mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.val.gz
Done ✔
Concatenating 16 parts → /mnt/disk3Tb/exported-slt-datasets/ISL-MERGED.pami0.train.gz
Done ✔


## Merge of pre-exported datasets → MERGE.pami0.{train,val,test}

In [3]:
# ================================================================
# Merge of pre-exported datasets → MERGE.pami0.{train,val,test}
# ================================================================
import gzip
import pickle
from pathlib import Path
from tqdm import tqdm

# Directory where the .pami0.* files live
OUTPUT_DIR = Path(OUTPUT_DIR)        # usa la misma variable del notebook; ajusta si fuera necesario

DATASETS   = ["GSL", "ISL", "LSAT", "RWTH_PHOENIX_2014T"]   # phoenix14t.* se ignoran
SPLITS     = ["train", "val", "test"]
BUFFER_LEN = 5_000        # nº de ejemplos a escribir por chunk (ajusta a tu RAM)

def stream_samples(path: Path):
    """Yield every sample stored (list o unitario) inside a gzip-pickle file."""
    with gzip.open(path, "rb") as f:
        while True:
            try:
                obj = pickle.load(f)
            except EOFError:
                break
            if isinstance(obj, list):
                yield from obj
            else:
                yield obj

def merge_split(split: str):
    out_path = OUTPUT_DIR / f"MERGE.pami0.{split}"
    buffer = []
    with gzip.open(out_path, "wb") as fout:
        for ds in DATASETS:
            in_path = OUTPUT_DIR / f"{ds}.pami0.{split}"
            print(f"→ adding {in_path}")
            for sample in stream_samples(in_path):
                buffer.append(sample)
                if len(buffer) >= BUFFER_LEN:
                    pickle.dump(buffer, fout, protocol=pickle.HIGHEST_PROTOCOL)
                    buffer.clear()
        if buffer:                                 # último resto
            pickle.dump(buffer, fout, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"✓ wrote {out_path}")

for split in SPLITS:
    merge_split(split)

print("🚀 MERGE.pami0.{train,val,test} listos en", OUTPUT_DIR)

→ adding /mnt/disk3Tb/exported-slt-datasets/GSL.pami0.train
→ adding /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.train
→ adding /mnt/disk3Tb/exported-slt-datasets/LSAT.pami0.train
→ adding /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.train
✓ wrote /mnt/disk3Tb/exported-slt-datasets/MERGE.pami0.train
→ adding /mnt/disk3Tb/exported-slt-datasets/GSL.pami0.val
→ adding /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.val
→ adding /mnt/disk3Tb/exported-slt-datasets/LSAT.pami0.val
→ adding /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.val
✓ wrote /mnt/disk3Tb/exported-slt-datasets/MERGE.pami0.val
→ adding /mnt/disk3Tb/exported-slt-datasets/GSL.pami0.test
→ adding /mnt/disk3Tb/exported-slt-datasets/ISL.pami0.test
→ adding /mnt/disk3Tb/exported-slt-datasets/LSAT.pami0.test
→ adding /mnt/disk3Tb/exported-slt-datasets/RWTH_PHOENIX_2014T.pami0.test
✓ wrote /mnt/disk3Tb/exported-slt-datasets/MERGE.pami0.test
🚀 MERGE.pami0.{train,val,test} listos en /mnt/disk3Tb/exported-s